<div style="background-color:#f0f8ff; border-radius:8px; padding:12px; text-align:center;">

# **PRACTICA 2: Árboles y Ensembles**

</div>

*Aprendizaje Automático*

---

**Grupo:** G-7312  
**Número de pareja:** 01  
**Miembros:**  
- Leire Bernárdez Vázquez  
- Carmen Reiné Rueda

---

### **Importaciones**

---

### **1. Clasificación mediante árboles de clasificación**

1. Cargue los datos del problema

2. Separe los datos en training (50 %) y test (50 %)

3. Utilice la clase DecisionTreeClassifier de Sklearn para resolver el problema
y d ́e un resultado de test usando la m ́etrica que considere oportuna

4. ¿Había missing values en nuestro problema? ¿Qué efecto tendría en los  ́arboles no completar los missing values ?

5. ¿Con qué parémetro podr ́ıa controlarse la profundidad del ́arbol? ¿Qué ocurre si un ́arbol se inicializa sin dar un valor concreto a ese parámetro? ¿Crees que sería una buena práctica no darle ningún valor?

6. ¿Son sensibles los ́arboles al desequilibrio entre clases?

7. ¿Existe desequilibrio entre las clases de nuestro problema? ¿Con qué parámetro de la clase DecisionTreeClassifier podrías corregirlo?

8. ¿Cuál es la diferencia entre los parámetros min samples split y min samples leaf ?

9. ¿Para qué sirve el parémetro criterion, qué valores puede tomar y qué significa o representa cada uno de esos valores?

10. El ́arbol entrenado, ¿realiza splits utilizando todas las variables disponibles?

11. ¿Se le ocurre alguna forma de crear un selector de atributos a partir de un árbol de clasificación?

---

### **2. Regresión mediante árboles de regresión**

---

### **3. Ensembles**